In [105]:
from decimal import Decimal, ROUND_DOWN
from typing import List
import operator
import csv
from collections import defaultdict
from functools import cmp_to_key
import numpy as np
import os
import sys
import time
import math
import sys
import time
import copy
import bisect

In [106]:
def binarySearch(arr, x):
    low, high = 0, len(arr) - 1

    while low <= high:
        mid = (high + low) // 2
        if arr[mid] < x:
            low = mid + 1
        elif arr[mid] > x:
            high = mid - 1
        else:
            return mid
    return -1

In [107]:
class ArraysAlgos:
    @staticmethod
    def cloneItemSetMinusOneItem(itemset, itemToRemove):
        return [item for item in itemset if item != itemToRemove]

    @staticmethod
    def cloneItemSetMinusAnItemset(itemset, itemsetToNotKeep):
        itemsetToNotKeepSet = set(itemsetToNotKeep)
        return [item for item in itemset if item not in itemsetToNotKeepSet]

    @staticmethod
    def allTheSameExceptLastItem(itemset1, itemset2):
        return itemset1[:-1] == itemset2[:-1]

    @staticmethod
    def concatenate(prefix, suffix):
        return prefix + suffix

    @staticmethod
    def intersectTwoSortedArrays(array1, array2):
        pos1, pos2 = 0, 0
        newArray = []
        while pos1 < len(array1) and pos2 < len(array2):
            if array1[pos1] == array2[pos2]:
                newArray.append(array1[pos1])
                pos1 += 1
                pos2 += 1
            elif array1[pos1] < array2[pos2]:
                pos1 += 1
            else:
                pos2 += 1
        return newArray

    @staticmethod
    def containsOrEquals(itemset1, itemset2):
        return all(item in itemset1 for item in itemset2)

    @staticmethod
    def containsLEX(itemset, item, maxItemInArray):
        return item in itemset if item <= maxItemInArray else False

    @staticmethod
    def sameAs(itemset1, itemset2, posRemoved):
        itemset2Adjusted = itemset2[:posRemoved] + itemset2[posRemoved+1:]
        return 0 if itemset1 == itemset2Adjusted else (-1 if itemset1 < itemset2Adjusted else 1)

    @staticmethod
    def includedIn(itemset1, itemset2):
        return all(item in itemset2 for item in itemset1)

    @staticmethod
    def containsLEXPlus(itemset, item):
        return any(item <= x for x in itemset)

    @staticmethod
    def contains(itemset, item):
        return item in itemset

    @staticmethod
    def comparatorItemsetSameSize(itemset1, itemset2):
        for i in range(len(itemset1)):
            if itemset1[i] != itemset2[i]:
                return -1 if itemset1[i] < itemset2[i] else 1
        return 0

    @staticmethod
    def appendIntegerToArray(array, integer):
        return array + [integer]

    @staticmethod
    def convertStringArrayToDoubleArray(tokens):
        return [float(token) for token in tokens]

    @staticmethod
    def isSubsetOf(itemset1, itemset2):
        return set(itemset1).issubset(set(itemset2))
    
    @staticmethod
    def concatenate(itemset1, itemset2):
        return itemset1 + itemset2

In [108]:
class AbstractItemset:
    def __init__(self):
        pass

    def size(self):
        raise NotImplementedError

    def __str__(self):
        raise NotImplementedError

    def print(self):
        print(str(self))

    def getAbsoluteSupport(self):
        raise NotImplementedError

    def getRelativeSupport(self, nbObject):
        raise NotImplementedError

    def getRelativeSupportAsString(self, nbObject):
        frequency = self.getRelativeSupport(nbObject)
        return f"{frequency:.5f}"

    def contains(self, item):
        raise NotImplementedError

In [109]:
class AbstractOrderedItemset(AbstractItemset):
    def __init__(self):
        super().__init__()

    def getAbsoluteSupport(self):
        raise NotImplementedError

    def size(self):
        raise NotImplementedError

    def get(self, position):
        raise NotImplementedError

    def getLastItem(self):
        return self.get(self.size() - 1)

    def __str__(self):
        return " ".join(str(self.get(i)) for i in range(self.size())) or "EMPTYSET"

    def getRelativeSupport(self, nbObject):
        return self.getAbsoluteSupport() / nbObject

    def contains(self, item):
        return any(self.get(i) == item for i in range(self.size()))

    def containsAll(self, itemset2):
        return all(any(self.get(i) == itemset2.get(j) for i in range(self.size())) for j in range(itemset2.size()))

    def isEqualTo(self, itemset2):
        if self.size() != itemset2.size():
            return False
        return all(self.get(i) == itemset2.get(i) for i in range(itemset2.size()))

    def allTheSameExceptLastItem(self, itemset2):
        if self.size() != itemset2.size():
            return False
        return all(self.get(i) == itemset2.get(i) for i in range(self.size() - 1))

In [110]:
class Itemset(AbstractOrderedItemset):
    def __init__(self, *args):
        self.itemset = []
        self.support = 0

        if args and isinstance(args[0], list):
            self.itemset = args[0]
            if len(args) > 1 and isinstance(args[1], int):
                self.support = args[1]
        elif args:
            self.itemset = [args[0]]
            if len(args) > 1 and isinstance(args[1], int):
                self.support = args[1]

    def getItems(self):
        return self.itemset

    def getAbsoluteSupport(self):
        return self.support

    def size(self):
        return len(self.itemset)

    def get(self, position):
        return self.itemset[position]

    def setAbsoluteSupport(self, support):
        self.support = support

    def increaseTransactionCount(self):
        self.support += 1

    def cloneItemSetMinusOneItem(self, itemToRemove):
        newItemset = [item for item in self.itemset if item != itemToRemove]
        return Itemset(newItemset)

    def cloneItemSetMinusAnItemset(self, itemsetToNotKeep):
        newItemset = [item for item in self.itemset if item not in itemsetToNotKeep.getItems()]
        return Itemset(newItemset)

    def intersection(self, itemset2):
        intersection = ArraysAlgos.intersectTwoSortedArrays(self.getItems(), itemset2.getItems())
        return Itemset(intersection)

    def hashCode(self):
        return hash(tuple(self.itemset))

In [111]:
class Itemsets:
    def __init__(self, name):
        self.levels = [[]]  # Initialize with one empty level
        self.itemsetsCount = 0
        self.name = name

    def printItemsets(self, nbObject):
        print(f" ------- {self.name} -------")
        patternCount = 0

        for levelCount, level in enumerate(self.levels):
            print(f"  L{levelCount} ")
            for itemset in level:
                support = itemset.getAbsoluteSupport()
                print(f"  pattern {patternCount}: {itemset} support :  {support}")
                patternCount += 1

        print(" --------------------------------")

    def addItemset(self, itemset, k):
        # Ensure the levels list is long enough to accommodate the new itemset
        if len(self.levels) <= k:
            self.levels.extend([[] for _ in range(k + 1 - len(self.levels))])

        self.levels[k].append(itemset)
        self.itemsetsCount += 1

    def getLevels(self):
        return self.levels

    def getItemsetsCount(self):
        return self.itemsetsCount

    def setName(self, newName):
        self.name = newName

    def decreaseItemsetCount(self):
        if self.itemsetsCount > 0:
            self.itemsetsCount -= 1

In [112]:
class FPNode:
    def __init__(self):
        self.itemID = -1
        self.counter = 1
        self.parent = None
        self.childs = []
        self.nodeLink = None

    def getChildWithID(self, id):
        return next((child for child in self.childs if child.itemID == id), None)

    def toString(self, indent=""):
        output = f"{self.itemID} (count={self.counter})\n"
        newIndent = indent + "   "
        output += ''.join(newIndent + child.toString(newIndent) for child in self.childs)
        return output

    def __str__(self):
        return str(self.itemID)

In [113]:
class FPTree:
    def __init__(self):
        self.headerList = None
        self.mapItemNodes = {}
        self.mapItemLastNode = {}
        self.root = FPNode()

    def addTransaction(self, transaction):
        currentNode = self.root
        for item in transaction:
            child = currentNode.getChildWithID(item)
            if child is None:
                newNode = FPNode()
                newNode.itemID = item
                newNode.parent = currentNode
                currentNode.childs.append(newNode)  # Use updated attribute name
                currentNode = newNode
                self.fixNodeLinks(item, newNode)
            else:
                child.counter += 1
                currentNode = child

    def fixNodeLinks(self, item, newNode):
        lastNode = self.mapItemLastNode.get(item)
        if lastNode:
            lastNode.nodeLink = newNode
        self.mapItemLastNode[item] = newNode
        if item not in self.mapItemNodes:
            self.mapItemNodes[item] = newNode

    def addPrefixPath(self, prefixPath, mapSupportBeta, relativeMinsupp):
        self._addPath(prefixPath, mapSupportBeta, relativeMinsupp, lambda support, pathLength: support >= relativeMinsupp)

    def addPrefixPathGRGrowth(self, prefixPath, mapSupportBeta, relativeMinsupp):
        self._addPath(prefixPath, mapSupportBeta, relativeMinsupp, lambda support, pathLength: support >= relativeMinsupp and support < pathLength)

    def _addPath(self, prefixPath, mapSupportBeta, relativeMinsupp, condition):
        pathCount = prefixPath[0].counter
        currentNode = self.root
        for i in range(len(prefixPath) - 1, 0, -1):
            pathItem = prefixPath[i]
            support = mapSupportBeta[pathItem.itemID]
            if condition(support, len(prefixPath)):
                child = currentNode.getChildWithID(pathItem.itemID)
                if child is None:
                    newNode = FPNode()
                    newNode.itemID = pathItem.itemID
                    newNode.parent = currentNode
                    newNode.counter = pathCount
                    currentNode.childs.append(newNode)
                    currentNode = newNode
                    self.fixNodeLinks(pathItem.itemID, newNode)
                else:
                    child.counter += pathCount
                    currentNode = child

    def createHeaderList(self, mapSupport):
        self.headerList = sorted(self.mapItemNodes.keys(), key=lambda id: (mapSupport[id], id), reverse=True)

    def __str__(self):
        return f"F HeaderList: {self.headerList}\n{self.root.toString('')}"

In [114]:
class AlgoFPGrowth:
    def __init__(self):
        self.startTimestamp = 0
        self.endTime = 0
        self.transactionCount = 0
        self.itemsetCount = 0
        self.minSupportRelative = 0
        self.writer = None
        self.patterns = None
        self.BUFFERS_SIZE = 2000
        self.itemsetBuffer = None
        self.fpNodeTempBuffer = None
        self.itemsetOutputBuffer = None
        self.maxPatternLength = 1000
        self.minPatternLength = 0

    def runAlgorithm(self, input, output, minsupp):
        self.startTimestamp = time.time()
        self.itemsetCount = 0
        if output is None:
            self.writer = None
            self.patterns = Itemsets("FREQUENT ITEMSETS")
        else:
            self.patterns = None
            self.writer = open(output, 'w')
            self.itemsetOutputBuffer = [0] * self.BUFFERS_SIZE
        mapSupport = self.scanDatabaseToDetermineFrequencyOfSingleItems(input)
        self.minSupportRelative = math.ceil(minsupp * self.transactionCount)
        tree = FPTree()
        with open(input, 'r') as reader:
            for line in reader:
                if line.strip() == '' or line[0] == '#' or line[0] == '%' or line[0] == '@':
                    continue
                lineSplited = line.split()
                transaction = [int(itemString) for itemString in lineSplited if mapSupport[int(itemString)] >= self.minSupportRelative]
                transaction.sort(key=lambda x: (mapSupport[x], x), reverse=True)
                tree.addTransaction(transaction)
        tree.createHeaderList(mapSupport)
        if tree.headerList:
            self.itemsetBuffer = [0] * self.BUFFERS_SIZE
            self.fpNodeTempBuffer = [None] * self.BUFFERS_SIZE
            self.fpgrowth(tree, self.itemsetBuffer, 0, self.transactionCount, mapSupport)
        if self.writer:
            self.writer.close()
        self.endTime = time.time()
        return self.patterns

    def fpgrowth(self, tree, prefix, prefixLength, prefixSupport, mapSupport):
        if prefixLength == self.maxPatternLength:
            return
        singlePath = True
        position = 0
        if len(tree.root.childs) > 1:
            singlePath = False
        else:
            currentNode = tree.root.childs[0]
            while True:
                if len(currentNode.childs) > 1:
                    singlePath = False
                    break
                self.fpNodeTempBuffer[position] = currentNode
                position += 1
                if len(currentNode.childs) == 0:
                    break
                currentNode = currentNode.childs[0]
        if singlePath:
            self.saveAllCombinationsOfPrefixPath(self.fpNodeTempBuffer, position, prefix, prefixLength)
        else:
            for i in range(len(tree.headerList) - 1, -1, -1):
                item = tree.headerList[i]
                support = mapSupport[item]
                prefix[prefixLength] = item
                betaSupport = prefixSupport if prefixSupport < support else support
                self.saveItemset(prefix, prefixLength + 1, betaSupport)
                if prefixLength + 1 < self.maxPatternLength:
                    prefixPaths = []
                    path = tree.mapItemNodes[item]
                    mapSupportBeta = {}
                    while path is not None:
                        if path.parent.itemID != -1:
                            prefixPath = []
                            prefixPath.append(path)
                            pathCount = path.counter
                            parent = path.parent
                            while parent.itemID != -1:
                                prefixPath.append(parent)
                                if parent.itemID not in mapSupportBeta:
                                    mapSupportBeta[parent.itemID] = pathCount
                                else:
                                    mapSupportBeta[parent.itemID] += pathCount
                                parent = parent.parent
                            prefixPaths.append(prefixPath)
                        path = path.nodeLink
                    treeBeta = FPTree()
                    for prefixPath in prefixPaths:
                        treeBeta.addPrefixPath(prefixPath, mapSupportBeta, self.minSupportRelative)
                    if len(treeBeta.root.childs) > 0:
                        treeBeta.createHeaderList(mapSupportBeta)
                        self.fpgrowth(treeBeta, prefix, prefixLength + 1, betaSupport, mapSupportBeta)

    def saveAllCombinationsOfPrefixPath(self, fpNodeTempBuffer, position, prefix, prefixLength):
        support = 0
        for i in range(1, 1 << position):
            newPrefixLength = prefixLength
            for j in range(position):
                isSet = (i & (1 << j)) > 0
                if isSet:
                    if newPrefixLength == self.maxPatternLength:
                        continue
                    prefix[newPrefixLength] = fpNodeTempBuffer[j].itemID
                    support = fpNodeTempBuffer[j].counter
                    newPrefixLength += 1
            self.saveItemset(prefix, newPrefixLength, support)

    def scanDatabaseToDetermineFrequencyOfSingleItems(self, input):
        mapSupport = defaultdict(int)
        with open(input, 'r') as reader:
            for line in reader:
                if line.strip() == '' or line[0] in '#%@':
                    continue
                for item in map(int, line.split()):
                    mapSupport[item] += 1
                self.transactionCount += 1
        return dict(mapSupport)

    def saveItemset(self, itemset, itemsetLength, support):
        if itemsetLength < self.minPatternLength:
            return
        self.itemsetCount += 1
        if self.writer is not None:
            itemsetOutputBuffer = sorted(itemset[:itemsetLength])
            buffer = ' '.join(str(x) for x in itemsetOutputBuffer) + " #SUP: " + str(support)
            self.writer.write(buffer + '\n')
        else:
            itemsetArray = sorted(itemset[:itemsetLength])
            itemsetObj = Itemset(itemsetArray, support)
            self.patterns.addItemset(itemsetObj, itemsetLength)


    def printStats(self):
        print("=============  FP-GROWTH 2.42 - STATS =============")
        temps = self.endTime - self.startTimestamp
        print(" Transactions count from database : ", self.transactionCount)
        print(" Frequent itemsets count : ", self.itemsetCount)
        print(" Total time ~ ", temps, " ms")
        print("===================================================")

    def getDatabaseSize(self):
        return self.transactionCount

    def setMaximumPatternLength(self, length):
        self.maxPatternLength = length

    def setMinimumPatternLength(self, minPatternLength):
        self.minPatternLength = minPatternLength

In [115]:
class Rule:
    def __init__(self, itemset1, itemset2, coverage, transactionCount, confidence):
        self.itemset1 = itemset1
        self.itemset2 = itemset2
        self.coverage = coverage
        self.transactionCount = transactionCount
        self.confidence = confidence

    def getRelativeSupport(self, databaseSize):
        return self.transactionCount / databaseSize

    def getAbsoluteSupport(self):
        return self.transactionCount

    def getConfidence(self):
        return self.confidence

    def getCoverage(self):
        return self.coverage

    def print(self):
        print(self.toString())

    def toString(self):
        buffer = []

        for i in range(len(self.itemset1)):
            buffer.append(str(self.itemset1[i]))
            if i != len(self.itemset1) - 1:
                buffer.append(" ")

        buffer.append(" ==> ")

        for i in range(len(self.itemset2)):
            buffer.append(str(self.itemset2[i]))
            buffer.append(" ")

        return "".join(buffer)

    def getItemset1(self):
        return self.itemset1

    def getItemset2(self):
        return self.itemset2

In [116]:
class AssocRule(Rule):
    def __init__(self, itemset1, itemset2, supportAntecedent, transactionCount, confidence, lift):
        super().__init__(itemset1, itemset2, supportAntecedent, transactionCount, confidence)
        self.lift = lift

    def __str__(self):
        # Leverage the string representation from the parent class and append lift information
        parent_str = super().__str__()
        return f"{parent_str} | Lift: {self.lift}"

In [117]:
class AssocRules:
    def __init__(self, name):
        self.rules = []
        self.name = name

    def sortByConfidence(self):
        self.rules.sort(key=lambda rule: rule.confidence, reverse=True)

    def printRules(self, databaseSize):
        print(f" ------- {self.name} -------")
        for i, rule in enumerate(self.rules):
            print(f"  rule {i}:  {rule.toString()} support :  {rule.getRelativeSupport(databaseSize)} ({rule.getAbsoluteSupport()}/{databaseSize}) confidence :  {rule.getConfidence()}")
            print("")
        print(" --------------------------------")

    def printRulesWithLift(self, databaseSize):
        print(f" ------- {self.name} -------")
        for i, rule in enumerate(self.rules):
            print(f"  rule {i}:  {rule}")
            print(f"support :  {rule.getRelativeSupport(databaseSize)} ({rule.getAbsoluteSupport()}/{databaseSize})")
            print(f"confidence :  {rule.getConfidence()}")
            print(f" lift :  {rule.getLift()}")
            print("")
        print(" --------------------------------")

    def __str__(self, databaseSize):
        buffer = f" ------- {self.name} -------\n"
        for i, rule in enumerate(self.rules):
            buffer += f"   rule {i}:  {rule}\n"
            buffer += f"support :  {rule.getRelativeSupport(databaseSize)} ({rule.getAbsoluteSupport()}/{databaseSize})\n"
            buffer += f"confidence :  {rule.getConfidence()}\n"
        return buffer

    def addRule(self, rule):
        self.rules.append(rule)

    def getRulesCount(self):
        return len(self.rules)

    def getRules(self):
        return self.rules

In [118]:
def custom_compare(itemset1, itemset2):
    return ArraysAlgos.comparatorItemsetSameSize(itemset1.getItems(), itemset2.getItems())

class AlgoAgrawalFaster94:
    def __init__(self):
        self.patterns = None
        self.rules = None
        self.writer = None
        self.startTimestamp = 0
        self.endTimestamp = 0
        self.ruleCount = 0
        self.databaseSize = 0
        self.maxConsequentLength = sys.maxsize
        self.maxAntecedentLength = sys.maxsize
        self.minconf = 0
        self.minlift = 0
        self.usingLift = True

    def runAlgorithm(self, patterns, output, databaseSize, minconf, minlift=0):
        self.minconf = minconf
        self.minlift = 0
        self.usingLift = False
        return self.runAlgorithmHelper(patterns, output, databaseSize)

    def runAlgorithmHelper(self, patterns, output, databaseSize):
        if self.maxAntecedentLength < 1 or self.maxConsequentLength < 1:
            raise ValueError("The maximum length must be at least 1.")

        if output is None:
            self.writer = None
            self.rules = AssocRules("ASSOCIATION RULES")
        else:
            self.rules = None
            self.writer = open(output, "w")

        self.databaseSize = databaseSize
        self.startTimestamp = time.time()
        self.ruleCount = 0
        self.patterns = patterns
        for itemsetsSameSize in patterns.getLevels():
            itemsetsSameSize.sort(key=cmp_to_key(custom_compare))
        for k in range(2, len(patterns.getLevels())):
            for lk in patterns.getLevels()[k]:
                H1_for_recursion = []
                for item in lk.getItems():
                    itemsetHm_P_1 = [item]
                    if lk.size() - 1 <= self.maxAntecedentLength:
                        itemset_Lk_minus_hm_P_1 = ArraysAlgos.cloneItemSetMinusOneItem(lk.getItems(),item)
                        support = self.calculateSupport(itemset_Lk_minus_hm_P_1)
                        supportAsDouble = float(support)
                        conf = lk.getAbsoluteSupport() / supportAsDouble
                        if conf < self.minconf or conf == sys.maxsize:
                            continue
                        lift = 0
                        supportHm_P_1 = 0
                        if self.usingLift:
                            supportHm_P_1 = self.calculateSupport(itemsetHm_P_1)
                            term1 = float(lk.getAbsoluteSupport()) / self.databaseSize
                            term2 = supportAsDouble / self.databaseSize
                            term3 = float(supportHm_P_1) / self.databaseSize
                            lift = term1 / (term2 * term3)
                            if lift < self.minlift:
                                continue
                        self.saveRule(itemset_Lk_minus_hm_P_1, support, itemsetHm_P_1, supportHm_P_1, lk.getAbsoluteSupport(), conf, lift)
                    if 1 < self.maxConsequentLength:
                        H1_for_recursion.append(itemsetHm_P_1)
                self.apGenrules(k, 1, lk, H1_for_recursion)

        if self.writer is not None:
            self.writer.close()
        self.endTimestamp = time.time()
        return self.rules

    def apGenrules(self, k, m, lk, Hm):
        if k > m + 1:
            Hm_plus_1_for_recursion = []
            Hm_plus_1 = self.generateCandidateSizeK(Hm)
            for hm_P_1 in Hm_plus_1:
                if lk.size() - len(hm_P_1) <= self.maxAntecedentLength:
                    itemset_Lk_minus_hm_P_1 = ArraysAlgos.cloneItemSetMinusAnItemset(lk.getItems(), hm_P_1);
                    print("itemset_Lk_minus_hm_P_1",itemset_Lk_minus_hm_P_1)
                    support = self.calculateSupport(itemset_Lk_minus_hm_P_1)
                    supportAsDouble = float(support)
                    print("supportAsDouble",supportAsDouble)
                    conf = lk.getAbsoluteSupport() / supportAsDouble
                    if conf < self.minconf or conf == sys.maxsize:
                        continue
                    lift = 0
                    supportHm_P_1 = 0
                    if self.usingLift:
                        supportHm_P_1 = self.calculateSupport(hm_P_1)
                        term1 = float(lk.getAbsoluteSupport()) / self.databaseSize
                        term2 = supportAsDouble / self.databaseSize
                        lift = term1 / (term2 * (float(supportHm_P_1) / self.databaseSize))
                        if lift < self.minlift:
                            continue
                    self.saveRule(itemset_Lk_minus_hm_P_1, support, hm_P_1, supportHm_P_1, lk.getAbsoluteSupport(), conf, lift)
                if k != m + 1 and m + len(hm_P_1) <= self.maxConsequentLength:
                    Hm_plus_1_for_recursion.append(hm_P_1)
            self.apGenrules(k, m + 1, lk, Hm_plus_1_for_recursion)

    def calculateSupport(self, itemset):
        # Retrieve patterns of the same size as the itemset
        patternsOfSameSize = self.patterns.getLevels()[len(itemset)]
        low = 0
        high = len(patternsOfSameSize) - 1

        # Perform binary search to find the itemset
        while low <= high:
            mid = (low + high) >> 1  # Equivalent to dividing by 2
            currentItems = patternsOfSameSize[mid].getItems()
            comparisonResult = ArraysAlgos.comparatorItemsetSameSize(itemset, currentItems)

            if comparisonResult > 0:
                low = mid + 1
            elif comparisonResult < 0:
                high = mid - 1
            else:
                # Itemset found, return its absolute support
                return patternsOfSameSize[mid].getAbsoluteSupport()

        # Itemset not found, return 0
        return 0

    def generateCandidateSizeK(self, levelK_1):
        candidates = []

        for i in range(len(levelK_1)):
            for j in range(i + 1, len(levelK_1)):
                # Check if the first K-2 items are identical
                if levelK_1[i][:-1] == levelK_1[j][:-1]:
                    # Compare the last items and generate a new candidate
                    if levelK_1[i][-1] < levelK_1[j][-1]:
                        newItemset = levelK_1[i] + [levelK_1[j][-1]]
                    else:
                        newItemset = levelK_1[j] + [levelK_1[i][-1]]
                    candidates.append(newItemset)

        return candidates

    def saveRule(self, itemset1,supportItemset1, itemset2, supportItemset2, absoluteSupport, conf, lift):
        self.ruleCount += 1
        if self.writer is not None:
            buffer = ""
            for i in range(len(itemset1)):
                buffer += str(itemset1[i])
                if i != len(itemset1) - 1:
                    buffer += " "
            buffer += " ==> "
            for i in range(len(itemset2)):
                buffer += str(itemset2[i])
                if i != len(itemset2) - 1:
                    buffer += " "
            buffer += " #SUP: "
            buffer += str(absoluteSupport)
            buffer += " #CONF: "
            buffer += str(conf)
            if self.usingLift:
                buffer += " #LIFT: "
                buffer += str(lift)
            self.writer.write(buffer)
            self.writer.write("\n")
        else:
            self.rules.addRule(AssocRule(itemset1, itemset2, supportItemset1, absoluteSupport, conf, lift))

    def setMaxAntecedentLength(self, maxAntecedentLength):
        self.maxAntecedentLength = maxAntecedentLength

    def setMaxConsequentLength(self, maxConsequentLength):
        self.maxConsequentLength = maxConsequentLength


In [119]:
input = "contextIGB.txt"
output = "fptgrowth.txt"

maxConsequentLength = 40
maxAntecedentLength = 40

# STEP 1: Applying the FP-GROWTH algorithm to find frequent itemsets
minsup = 0.5
fpgrowth = AlgoFPGrowth()
fpgrowth.setMaximumPatternLength(maxAntecedentLength + maxConsequentLength)
patterns = fpgrowth.runAlgorithm(input, None, minsup)
databaseSize = fpgrowth.getDatabaseSize();

# STEP 2: Generating all rules from the set of frequent itemsets (based on Agrawal & Srikant, 94)
minconf = 0.6
algoAgrawal = AlgoAgrawalFaster94()
algoAgrawal.setMaxConsequentLength(maxConsequentLength);
algoAgrawal.setMaxAntecedentLength(maxAntecedentLength);
rules = algoAgrawal.runAlgorithm(patterns, None, databaseSize, minconf);
rules.printRules(databaseSize);
print("DATABASE SIZE ",databaseSize);

itemset_Lk_minus_hm_P_1 [4]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [2]
supportAsDouble 6.0
itemset_Lk_minus_hm_P_1 [1]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [5]
supportAsDouble 5.0
itemset_Lk_minus_hm_P_1 [2]
supportAsDouble 6.0
itemset_Lk_minus_hm_P_1 [1]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [5]
supportAsDouble 5.0
itemset_Lk_minus_hm_P_1 [4]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [1]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [5]
supportAsDouble 5.0
itemset_Lk_minus_hm_P_1 [3]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [2]
supportAsDouble 6.0
itemset_Lk_minus_hm_P_1 [5]
supportAsDouble 5.0
itemset_Lk_minus_hm_P_1 [4]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [2]
supportAsDouble 6.0
itemset_Lk_minus_hm_P_1 [4, 5]
supportAsDouble 3.0
itemset_Lk_minus_hm_P_1 [2, 5]
supportAsDouble 5.0
itemset_Lk_minus_hm_P_1 [2, 4]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [1, 5]
supportAsDouble 4.0
itemset_Lk_minus_hm_P_1 [1, 4]
supportAsDouble 3.0
itemset_Lk_minus_hm_P_1 [